In [ ]:
import os
import shutil
import yaml
import time
import json
import random
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
import seaborn as sns
import torch
from IPython.display import Image, clear_output
from PIL import Image
from tqdm.auto import tqdm
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
KAGGLE = False
VER = 'v4'
if KAGGLE:
    DATA_PATH = '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection'
    MDLS_PATH = f'/kaggle/input/vinbigdata-models-{VER}'
    CACHE_PATH = '/kaggle/working/cache'
    YOLO_DIR = '/kaggle/working/yolov5'
    SUBM_DIR = '/kaggle/working'
    shutil.copytree(
        '/kaggle/input/yolov5-official-v31-dataset/yolov5', 
        YOLO_DIR
    )
else:
    DATA_PATH = '/u01/mrorange/vinbigdata/data'
    MDLS_PATH = f'/u01/mrorange/vinbigdata/models_{VER}'
    CACHE_PATH = f'{DATA_PATH}/working/cache'
    YOLO_DIR = f'{DATA_PATH}/working/yolov5'
    SUBM_DIR = '/u01/mrorange/vinbigdata'
os.chdir(YOLO_DIR)
TEST_PATH = f'{DATA_PATH}/test/'
THRESHOLD = .5
with open(f'{MDLS_PATH}/params.json') as file:
    params = json.load(file)
print('loaded params:', params)
SIZE = params['img_size']

start_time = time.time()

In [ ]:
def read_xray(path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array        
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8) 
    return data

def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    img = Image.fromarray(array)
    if keep_ratio:
        img.thumbnail((size, size), resample)
    else:
        img = img.resize((size, size), resample)
    return img

def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]] * image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]] * image_height
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]] / 2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    return bboxes

In [ ]:
image_id = []
dim0 = []
dim1 = []
os.makedirs(CACHE_PATH, exist_ok=True)

for file in tqdm(os.listdir(TEST_PATH)):
    xray = read_xray(TEST_PATH + file)
    im = resize(xray, size=SIZE) # keep_ratio=True to have original aspect ratio
    im.save(CACHE_PATH + '/' + file.replace('dicom', 'png'))
    image_id.append(file.replace('.dicom', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
df = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1})

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
!/opt/anaconda3/envs/orange/bin/python detect.py --weights $MDLS_PATH/best.pt \
--img 640 \
--conf 0.15 \
--iou 0.4 \
--source $CACHE_PATH \
--save-txt --save-conf --exist-ok

In [ ]:
elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
image_ids = []
pred_strings = []
for file_path in tqdm(glob(f'{YOLO_DIR}/runs/detect/exp/labels/*txt')):
    image_id = file_path.split('/')[-1].split('.')[0]
    w, h = df.loc[df.image_id == image_id, ['dim1', 'dim0']].values[0]
    with open(file_path, 'r') as file:
        data = np.array(
            file.read().replace('\n', ' ').strip().split(' ')
        ).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]
    bboxes = list(
        np.round(
            np.concatenate(
                (
                    data[:, :2], 
                    np.round(yolo2voc(h, w, data[:, 2:]))
                ), axis=1
            ).reshape(-1), 1
        ).astype(str)
    )
    for idx in range(len(bboxes)):
        bboxes[idx] = str(int(float(bboxes[idx]))) if (idx % 6) != 1 else bboxes[idx]
    image_ids.append(image_id)
    pred_strings.append(' '.join(bboxes))
    
elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
os.chdir(SUBM_DIR)
shutil.rmtree(CACHE_PATH)
if KAGGLE:
    shutil.rmtree(YOLO_DIR)
print('temp folders deleted')

In [ ]:
pred_df = pd.DataFrame({'image_id': image_ids,
                        'PredictionString': pred_strings})
subm_df = pd.merge(
    df, 
    pred_df, 
    on='image_id', 
    how = 'left'
).fillna('14 1 0 0 1 1')
subm_df = subm_df[['image_id', 'PredictionString']]
subm_df.to_csv('submission.csv', index=False)
print('submission dataframe saved:', subm_df.shape)
    
elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

subm_df.tail()